## 머신러닝 실습

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 이용
- 프로세스 - 텍스트전처리 > 특성 벡터화 > 머신러닝 모델 구축,학습,평가
- 텍스트 전처리 - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석...
- 
##### 특성 벡터화/ 추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
- BoW, Word2ve 라이브러리(모듈) 존재
- 
##### LDA
- 문서에 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_train.txt, ratings_test.txt 다운로드
- id(리뷰번호), document(리뷰), label(감성분류클래스 0은 부정, 1은 긍정 감성)



In [ ]:
## 필요 라이브러리 사용등록
import pandas as pd
import numpy as np
import re

In [ ]:
# 모든 모듈/라이브러리는 __version__
pd.__version__

In [ ]:
## pandas 버전 상세정보
pd.show_versions()

In [ ]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [ ]:
dfNsmcTrain.head()

In [ ]:
dfNsmcTrain.info()

In [ ]:
## 결측치 확인
dfNsmcTrain[ dfNsmcTrain['document'].isnull() ]

In [ ]:
dfNsmcTrain = dfNsmcTrain[ dfNsmcTrain['document'].notnull() ]

In [ ]:
dfNsmcTrain.info()

In [ ]:
dfNsmcTrain['label'].value_counts()

In [ ]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ', x))

In [ ]:
dfNsmcTrain.tail()

In [ ]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [ ]:
dfNsmcTest.head()

In [ ]:
dfNsmcTest.info()

In [ ]:
dfNsmcTest[ dfNsmcTest['document'].isnull() ]

In [ ]:
dfNsmcTest = dfNsmcTest[ dfNsmcTest['document'].notnull() ]

In [ ]:
dfNsmcTest['label'].value_counts()

In [ ]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ', x))

In [ ]:
dfNsmcTest

In [ ]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[  dfNsmcTrain['document'] != ' ' ]

In [ ]:
dfNsmcTrain.info()

In [ ]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터가 554개 존재
dfNsmcTest = dfNsmcTest[  dfNsmcTest['document'] != ' ' ]

In [ ]:
dfNsmcTest.info()

##### 분석모델 구축
- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키의 Okt 클래스 사용
##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템등록정보 JAVA_HOME 설정
    - 프로그램 실행 : sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템변수 > 새로 만들기 버튼 클릭
    - JAVA_HOME 과 경로 추가 등록
- JPype1 - 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [ ]:
# Jpype1 설치
!pip install JPype1

In [ ]:
# koNLPy 설치
!pip install koNLPy

In [ ]:
import konlpy

In [ ]:
konlpy.__version__

In [ ]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
okt = Okt()

In [ ]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [ ]:
# 벡터화 시켜주는 객체
tfidf = TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)

In [ ]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

In [ ]:
nsmc_train_tfidf

In [ ]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)

##### 감성 분류모델 구축

In [ ]:
## 로지스틱회귀 라이브러리 등록
from sklearn.linear_model import LogisticRegression


In [ ]:
model = LogisticRegression(random_state = 0)

In [ ]:
## 감성 이진분류 모델 훈련
model.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

In [ ]:
# 로지스틱회귀 사용될 하이퍼파라미터 리스트
model.get_params()

In [ ]:
## 최적 예측모델을 찾기위한 작업
## 추가 라이브러리 사용등록
from sklearn.model_selection import GridSearchCV

In [ ]:
## 하이퍼파라미터 중 C(현재 1.0) 값을 변경하면서 최적예측 모델 찾기
params = {'C' : [3.0, 3.5, 4.0, 4.5, 5.0]}
testModel = GridSearchCV(model, param_grid=params, cv=3, scoring = 'accuracy', verbose=1)

In [ ]:
testModel.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

In [ ]:
testModel.best_params_

In [ ]:
testModel.best_score_

In [ ]:
# c = 3.5로 해서 bestModel을 생성
bestModel = testModel.best_estimator_

In [ ]:
dfNsmcTest.tail()

##### 검증(평가)용 데이터 벡터화

In [ ]:
nsmc_test_tfidf = tfidf.transform(dfNsmcTest['document'])

In [ ]:
# 예측결과
y_predict = bestModel.predict(nsmc_test_tfidf)

In [ ]:
# 감정분류 에측결과, 실제 라벨수 동일
len(y_predict), len(dfNsmcTest['label'])

In [ ]:
# 평가지표로 정확도 확인
from sklearn.metrics import accuracy_score

In [ ]:
# 베스트모델로 예측결과
accuracy_score(dfNsmcTest['label'], y_predict)

##### 베스트모델로 감성예측

In [ ]:
sentence = input('리뷰 입력 >>')

In [ ]:
sentence

In [ ]:
# 입력 문장에서 한글만 추출
st = re.compile(r'[가-힣]').findall(sentence)
st

In [ ]:
# 입력 문장에서 한글 외에 제거하고 다시 문장으로 할당
st = re.sub(r'[^가-힣|ㄱ-]+',' ',sentence)
st

In [ ]:
# 리스트로 형변환
listSt = [st]

In [ ]:
# 입력 텍스트의 벡터화
st_tfidf = tfidf.transform(listSt)
st_tfidf

In [ ]:
# 1은 긍정적인 감정, 0은 부정적인 감정
bestModel.predict(st_tfidf)

##### 결론
- 감정예측은 오류가 많을 수 있다.
- 많은 훈련과, 머신러닝을 딥러닝과 결합해서 정확도를 높여야 한다.